Lab03
=====

### 분류 프로젝트
지난 5일간 배운 내용을 바탕으로 자유롭게 분류 모델을 만들고 분석 보고서를 작성하시면 됩니다.

In [ ]:
import os
from os.path import join

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

train_datapath = join('data', 'raw_adult_train.txt')
test_datapath = join('data', 'raw_adult_test.txt')
columnpath = join('data','adult_attributes.txt')

columns = list()
with open(columnpath, 'r') as f:
    columns = f.read().split('\n')

## 1. 데이터 살펴보기 
이번에 사용할 데이터는 미국 시민들의 데이터를 분석하여 소득이 50K 이상인지 아닌지를 분류하는 것을 목표로하는 Adult data입니다.<br>
Adult 데이터는 15개의 컬럼을 가지고 있으며, 2개의 클래스로 나누어져 있습니다.<br> 
또한, 이미 학습에 사용할 Train 데이터와 검증에 사용할 Test 데이터로 분할이 되어있습니다.<br>
해당 데이터셋은 소득 컬럼을 이용해 분류에 사용할 수도 있고, 클러스터링, 회귀, 차원축소 등 여러가지 머신러닝 기법에 사용될 수 있습니다.<br><br>
### Featrue Description
1. age : 나이
2. workclass : 직군
3. fnlwgt : 정보를 기반으로 한 자체 인구 통계 점수
4. education : 학력 수준
5. education-num : 학력 수준을 수치로 나타낸 형태
6. marital-status : 혼인 상태
7. occupation : 직업
8. relationship : 가족 관계
9. race : 인종
10. sex : 성별
11. capital-gain : 자본 소득
12. capital-loss : 자본 손실 
13. hours-per-week : 주 근무시간
14. native-country : 출신 국가
15. income(target) : 소득이 50K를 넘는지에 대한 유무

DataFrame.head() 함수로 처음 5개의 데이터를 살펴볼 수 있습니다.

In [ ]:
raw_train_data = pd.read_csv(train_datapath, names = columns, na_values = " ?")
raw_train_data.head()

In [ ]:
raw_test_data = pd.read_csv(test_datapath, names = columns, na_values = " ?")
raw_test_data.head()

### 변수별 분포 확인해보기

데이터와 변수가 너무 많아 모든 변수를 시각화 하는데 오랜시간이 걸리므로 이미지로 대체하겠습니다.
![All_Distribution](./Images/Distribute.png)

### income(target) 컬럼 확인해보기
Train 데이터에 대해서, income이 >50K인 데이터가 전체의 75%이상을 차지하고 있는 것을 확인할 수 있습니다.<br>
이와 같은 이진 분류 문제에서는 구성한 모델이 모든 데이터에 대해 '0'이라고 추론하여도 75%의 정확도를 가지게 되므로, 모델 학습 시 주의할 필요가 있습니다.

![income](./Images/income.png)

## 2. 간단한 전처리

### 2.1 결측치 확인해보기
실습 수업에서는 배우지 않았지만, 해당 데이터의 결측치를 확인해 보겠습니다.

In [ ]:
print('Train Columns \t Number of NaN')
for c in raw_train_data.columns:
    n_nan = raw_train_data[c].isna().sum()
    if n_nan != 0 :
        print("{} \t {}".format(c,n_nan))

In [ ]:
print('Test Columns \t Number of NaN')
for c in raw_test_data.columns:
    n_nan = raw_test_data[c].isna().sum()
    if n_nan != 0 :
        print("{} \t {}".format(c,n_nan))

Train, Test 데이터 모두 직군, 직업, 출생국가 변수에서 결측치를 확인할 수 있었습니다. <br>
데이터의 결측상태가 심하지 않으므로, 결측치 데이터를 제외하기 보다는 다른 특성들을 활용하여 채우는 방향으로 진행하겠습니다.

### 2.2 결측치 채우기
직군과 직업은 학력과 어느정도 관계가 있다고 생각할 수 있습니다.<br>
따라서 결측치가 존재하는 직군, 직업 변수를 학력과 비교하여 채워보겠습니다.

#### 2.2.1 workclass 채우기
직군에서는 학력수준에 관계없이 'Private'가 주를 이루고 있는 것을 확인할 수 있습니다.<br>
이에 따라'workclass'에 존재하는 결측치를 'Private'로 채우도록 하겠습니다.

In [ ]:
pd.crosstab(raw_train_data['education-num'], raw_train_data['workclass']).T.style.background_gradient(cmap='summer_r')

In [ ]:
raw_train_data['workclass'] = raw_train_data['workclass'].replace(np.nan, 'Private')
raw_test_data['workclass'] = raw_test_data['workclass'].replace(np.nan, 'Private')

#### 2.2.2 native-country 채우기
해당 변수는 미국이 90%이상의 분포를 차지하고 있으므로, 미국으로 채우도록 하겠습니다.

In [ ]:
raw_train_data['native-country'] = raw_train_data['native-country'].replace(np.nan, 'United-States')
raw_test_data['native-country'] = raw_test_data['native-country'].replace(np.nan, 'United-States')

#### 2.2.3 occupation 채우기
2.1.1에서 실제 직군은 공개하지 않는 'Private'가 존재하지만, 직업은 모두 공개되어 있습니다.<br>
각각 학력마다 특정 직업에 데이터가 몰리는 것을 확인할 수 있었습니다.<br>
이에 따라 occupation 변수의 결측치를 각 학력에서 가장 많은 직업을 선택하여 채우도록 하겠습니다.

In [ ]:
crosstab = pd.crosstab(raw_train_data['education-num'], raw_train_data['occupation']).T
crosstab.style.background_gradient(cmap='summer_r')

In [ ]:
occu_list = list()
# 각 학력 수준 별 가장 많은 직업 리스트 만들기
for i in crosstab.columns:
    occu_list.append(np.argmax(crosstab.iloc[:,i-1]))

In [ ]:
for i in range(1,17):
    raw_train_data.loc[raw_train_data['education-num'] == i] = \
    raw_train_data.loc[raw_train_data['education-num'] == i].replace(np.nan, occu_list[i-1])
    
    raw_test_data.loc[raw_test_data['education-num'] == i] = \
    raw_test_data.loc[raw_test_data['education-num'] == i].replace(np.nan, occu_list[i-1])

#### 2.2.4 데이터가 제대로 채워졌는지 확인하기

In [ ]:
print('Train Columns \t Number of NaN')
for c in raw_train_data.columns:
    n_nan = raw_train_data[c].isna().sum()
    if n_nan != 0 :
        print("{} \t {}".format(c,n_nan))

In [ ]:
print('Test Columns \t Number of NaN')
for c in raw_test_data.columns:
    n_nan = raw_test_data[c].isna().sum()
    if n_nan != 0 :
        print("{} \t {}".format(c,n_nan))

### 2.3 Native-Country 전처리
출신국가의 경우 극도로 변수가 치우쳐져 있는 것을 확인할 수 있습니다.<br>
이러한 형태의 데이터 불균형 문제는 모델의 성능을 안좋게 하거나 편향된 결과를 낼 수 있으므로 전처리를 통해 불균형을 완화 해보겠습니다.<br>
이러한 데이터 불균형을 처리하는 좋은 방법으로 데이터의 추상화 레벨을 올려 데이터를 통합 하는 것이 하나의 방법이 될 수 있습니다.<br>
IMF에서 공개한 GDP 기준 상위 10개국에 대해 선진국으로 할당하고 나머지 국가를 하나의 변수로 할당하겠습니다.<br>
데이터에 존재하지 않은 국가는 리스트에서 제외하였습니다.

In [ ]:
adv_list = ['United-States', 'China', 'Japan', 'Germany', 'England', 'France', 'India', 'Italy',
           'Brazil', 'Canada']

new_train_native_country = list()
for d in raw_train_data['native-country']:
    if d in adv_list:
        new_train_native_country.append('Adv')
    else:
        new_train_native_country.append('Under')
raw_train_data['native-country'] = new_train_native_country

In [ ]:
new_test_native_country = list()
for d in raw_test_data['native-country']:
    if d in adv_list:
        new_test_native_country.append('Adv')
    else:
        new_test_native_country.append('Under')
raw_test_data['native-country'] = new_test_native_country

전처리한 결과 입니다.
![New_counry](./Images/New_Country.png)

### 2.4 라벨(income) 정리
income이 50K가 넘는 경우 1, 넘지 않는 경우 0으로 변경하겠습니다.<br>
그리고 학습에 사용하기위해 다른 변수로 저장하고 데이터에서 제거하겠습니다.

In [ ]:
raw_train_data['income'] = raw_train_data['income'].replace(' >50K', 1)
raw_test_data['income']  = raw_test_data['income'].replace(' >50K.', 1)
raw_train_data['income'] = raw_train_data['income'].replace(' <=50K', 0)
raw_test_data['income']  = raw_test_data['income'].replace(' <=50K.', 0)

train_labels = raw_train_data['income']
test_labels = raw_test_data['income']

In [ ]:
del raw_train_data['income']
del raw_test_data['income']

In [ ]:
raw_train_data.head()

In [ ]:
raw_test_data.head()

### 2.5 fnlwgt 변수와 education 변수 삭제
fnlwgt 변수는 데이터를 제공하는 기관에서 만든 지표입니다. 따라서 우리의 모델에서는 적용하지 않겠습니다.<br>
그리고 교육 수준을 나타내는 education 변수는 education-num과 강한 선형 관계를 나타내므로 제거했습니다.

In [ ]:
del raw_train_data['fnlwgt']
del raw_test_data['fnlwgt']
del raw_train_data['education-num']
del raw_test_data['education-num']

### 2.6. 범주형 데이터 정리
범주형 변수의 경우 ont-hot encoding을 통해 서로 독립적인 벡터로 변경해줍니다.<br>
pandas에서는 pandas.get_dummies() 함수를 통해 쉽게 변경할 수 있습니다.

In [ ]:
train_data = pd.get_dummies(raw_train_data, drop_first=True)
test_data = pd.get_dummies(raw_test_data, drop_first=True)

In [ ]:
train_data.head()

## 3. Sklearn으로 분류 모델링하기
지금까지 데이터를 모델에 넣기 좋게 전처리를 진행하였습니다. 이제 모델링을 시작해보겠습니다.<br>
이번 프로젝트에서는 실습에서 배운 k-NN, Decision Tree, Logistic Regression 분류기를 사용합니다.<br>
심화 내용으로 앙상블 이라는 여러개의 모델을 하나의 모델로 만드는 분류기에 대해서도 맛보겠습니다.<br><br>

### 3.1 k-NN 분류기

In [ ]:
#k-NN 모델 생성
clf1 = KNeighborsClassifier(n_neighbors = 3)
#학습
clf1.fit(train_data, train_labels)

훈련 데이터로 학습한 모델의 훈련 데이터 정확도와 검증 데이터 정확도를 살펴보겠습니다.

In [ ]:
train_acc = clf1.score(train_data, train_labels)
test_acc  = clf1.score(test_data,  test_labels)
print('훈련 데이터 정확도 : {:2f}% 검증 데이터 정확도 : {:2f}%'.format(train_acc*100, test_acc*100))

In [ ]:
# ROC 곡선 계산
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(test_labels, clf1.predict_proba(test_data)[:, 1])
    roc_auc[i] = auc(fpr[i], tpr[i])

# ROC 곡선 영역인 auc 계산
fpr["micro"], tpr["micro"], _ = roc_curve(test_labels.ravel(), clf1.predict_proba(test_data)[:,1].ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

### 3.2 Decision Tree 분류기

In [ ]:
#Decision Tree 모델 생성
clf2 = DecisionTreeClassifier(criterion = 'entropy', max_depth = 10, random_state=2019)
#학습
clf2.fit(train_data, train_labels)

In [ ]:
train_acc = clf2.score(train_data, train_labels)
test_acc  = clf2.score(test_data,  test_labels)
print('훈련 데이터 정확도 : {:2f}% 검증 데이터 정확도 : {:2f}%'.format(train_acc*100, test_acc*100))

In [ ]:
# ROC 곡선 계산
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(test_labels, clf2.predict_proba(test_data)[:, 1])
    roc_auc[i] = auc(fpr[i], tpr[i])

# ROC 곡선 영역인 auc 계산
fpr["micro"], tpr["micro"], _ = roc_curve(test_labels.ravel(), clf2.predict_proba(test_data)[:,1].ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Decision Tree는 각 변수의 중요도를 살펴볼 수 있으니 변수 중요도도 살펴보겠습니다.

In [ ]:
fi_dict = dict()
for c in raw_train_data.columns:
    fi_dict[c] = 0
    for i, d_c in zip(clf2.feature_importances_, train_data.columns):
        if c in d_c  :
            fi_dict[c] += i

In [ ]:
fi = np.array(sorted(list(fi_dict.items()), key = lambda x : x[1], reverse=True))
fi_value = np.array(fi[:,1], np.float)
fi_name = fi[:,0]

In [ ]:
plt.bar(np.arange(len(fi)), fi_value)
plt.xticks(range(12), fi_name, rotation=90)
plt.show()

결혼 유무, 나이, 자본 소득, 직업, 주당 근무시간, 직군... 순서로 모델이 변수 중요도를 보여주었습니다.<br>
직관적으로 이해하기에도 납득이 가능한 정도의 변수로 보입니다.

### 3.3 Logistic Regression 분류기

In [ ]:
#Logistic Regression 모델 생성
clf3 = LogisticRegression(random_state=2019)
#학습
clf3.fit(train_data, train_labels)

In [ ]:
train_acc = clf3.score(train_data, train_labels)
test_acc  = clf3.score(test_data,  test_labels)
print('훈련 데이터 정확도 : {:2f}% 검증 데이터 정확도 : {:2f}%'.format(train_acc*100, test_acc*100))

In [ ]:
# ROC 곡선 계산
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(test_labels, clf3.predict_proba(test_data)[:, 1])
    roc_auc[i] = auc(fpr[i], tpr[i])

# ROC 곡선 영역인 auc 계산
fpr["micro"], tpr["micro"], _ = roc_curve(test_labels.ravel(), clf3.predict_proba(test_data)[:,1].ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

### 3.4 앙상블 모델 (심화)
앞에서 생성한 3개의 분류 모델을 합쳐 데이터의 결과를 추론하는 앙상블 모델을 사용해보도록 하겠습니다.

In [ ]:
#앙상블 모델 생성
ensemble_clf = VotingClassifier([('k-NN',clf1), ('DT', clf2), ('LR', clf3)])
#학습
ensemble_clf.fit(train_data, train_labels)

In [ ]:
train_acc = ensemble_clf.score(train_data, train_labels)
test_acc  = ensemble_clf.score(test_data,  test_labels)
print('훈련 데이터 정확도 : {:2f}% 검증 데이터 정확도 : {:2f}%'.format(train_acc*100, test_acc*100))

In [ ]:
# ROC 곡선 계산
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(2):
    fpr[i], tpr[i], _ = roc_curve(test_labels, clf3.predict_proba(test_data)[:, 1])
    roc_auc[i] = auc(fpr[i], tpr[i])

# ROC 곡선 영역인 auc 계산
fpr["micro"], tpr["micro"], _ = roc_curve(test_labels.ravel(), clf3.predict_proba(test_data)[:,1].ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr[1], tpr[1], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[1])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

## 4. 결론
기존의 데이터에 존재하던 결측치 처리와 IMF에서 제공하는 외부 데이터를 사용해 클래스 불균형을 완화하고<br>
범주 형 데이터에 대해서는 One-Hot Encoding 처리를 하여 데이터를 전처리 했습니다.<br>
k-NN, Decision Tree, Logistic Regression을 통해 각각 모델의 정확도 및 ROC curve를 확인해 보았습니다.<br>
단일 모델에서는 Decision Tree가 가장 높은 검증 데이터 정확도와 완만한 ROC curve형태를 보여주었습니다.<br>
분류 모델의 심화 단계로, 3개의 단일 모델을 하나로 합쳐 앙상블 학습을 진행하는 모델을 사용해 조금 향상된 검증 데이터 정확도를 얻을 수 있었습니다.<br>
Decision Tree의 Feature Importance를 통해 우리가 사용한 데이터의 변수 중 어떤 변수가 트리 모델에서 중요하게 작용되었는지를 확인해 보았습니다.<br>
결혼 상태, 나이, 자본 소득, 직업, 주당 근무시간, 직군... 순서로 트리 모델이 변수 중요도를 뽑았는데, 직관적으로도 납득이 가는 수준에서 변수가 중요함을 알 수 있었습니다.<br>
조금 더 모델의 성능을 올리고자 한다면, 현재 데이터에서 해결하지 못한 변수내 불균형 문제를 해결하고 파생변수 등을 생성하여 더 좋은 데이터로 가공하는 것이 가장 합리적인 선택이 될 수 있습니다.<br>